University of Washington

CSE 493W: Wireless Communication, Spring 2023

Author: Amit Ferman


Transmits continuous carrier sinusoid on Hack RF One.

In [1]:
import SoapySDR
from SoapySDR import *
import numpy as np
import signal

In [2]:
# configure

args = dict(driver="hackrf")
sdr = SoapySDR.Device(args)

print("Antennas: ", sdr.listAntennas(SOAPY_SDR_RX, 0))
print("Gains: ", sdr.listGains(SOAPY_SDR_RX, 0))
print("Min/Max Freqs: ")
freqs = sdr.getFrequencyRange(SOAPY_SDR_RX, 0)
for freq in freqs:
    print(freq)

Antennas:  ('TX/RX',)
Gains:  ('LNA', 'AMP', 'VGA')
Min/Max Freqs: 
0, 7.25e+09


In [3]:
# receive some data to test receiver
rxStream = sdr.setupStream(SOAPY_SDR_RX, SOAPY_SDR_CF32)
sdr.activateStream(rxStream)
buff = np.array([0]*1024, np.complex64)
sr = sdr.readStream(rxStream, [buff], len(buff))
print("IQ data (first 2 samples): ", buff[0:2])
print("num samples or error code: ", sr.ret)
print("flags set by receive operation: ", sr.flags)
print("timestamp for receive buffer: ", sr.timeNs)
sdr.deactivateStream(rxStream)
sdr.closeStream(rxStream)

IQ data (first 2 samples):  [0.01574803+0.00787402j 0.01574803+0.00787402j]
num samples or error code:  1024
flags set by receive operation:  0
timestamp for receive buffer:  0


In [4]:
# transmit a continuous carrier sinusoid forever

tx_chan = 0
tx_sps = 10e6       # 10MHz  sampling rate
freq = 905e6        # 905Mhz center frequency
# tx_gain = 0       # 0db


sdr.setSampleRate(SOAPY_SDR_TX, tx_chan, tx_sps)
sdr.setFrequency(SOAPY_SDR_TX, tx_chan, freq)
# sdr.setGain(SoapySDR.SOAPY_SDR_TX, tx_chan, tx_gain)

tx_stream = sdr.setupStream(SOAPY_SDR_TX, SOAPY_SDR_CF32, [tx_chan])
sdr.activateStream(tx_stream)

tx_data = np.repeat(np.array(1 + 0j), 1024)

while True:
    sdr.writeStream(tx_stream, [tx_data], tx_data.size)

sdr.deactivateStream(tx_stream)
sdr.closeStream(tx_stream)